# **Music Recommendation system**

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data =  pd.read_csv('/content/drive/MyDrive/ spotify_millsongdata.csv')

In [ ]:
data.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
data.tail(5)

,artist,song,text
6995,Wanda Jackson,Lovin' Country Style,i like my lovin ' countri style i 'm onli in t...
6996,Tim Buckley,The Train,"ah , tell me mama ah , tell me darlin ' now th..."
6997,Point Of Grace,The River,god 's passion is an endless river white water...
6998,Proclaimers,Oh Jean,i 'd never been lucki with girl i confess do n...
6999,Rage Against The Machine,Autologic,c'mon ! go ... go ... go ... ugh .. go ... go ...


In [ ]:
data = data.sample(7000).drop('link',axis=1).reset_index(drop=True)

In [ ]:
data.head()

,artist,song,text
0,Elvis Presley,And The Grass Won't Pay No Mind,"Listen easy, you can hear God calling \nWalki..."
1,Judy Garland,Look For The Silver Lining,"As I wash my dishes, I'll be following a plan ..."
2,Meat Loaf,Blind Before I Stop,I'll go blind before I stop \n \n[Chorus] \...
3,Toto,Mrs. Johnson,Know these words are wrong to say \nNeed you ...
4,Michael Bolton,Survivor,"(You're a survivor, you're a survivor, survivo..."


In [ ]:
data['text']  = data['text'].str.lower().replace(r"\w\s", " ").replace(r"\n"," ",regex=True)

In [ ]:
data.head()

,artist,song,text
0,Elvis Presley,And The Grass Won't Pay No Mind,"listen easy, you can hear god calling walkin..."
1,Judy Garland,Look For The Silver Lining,"as i wash my dishes, i'll be following a plan ..."
2,Meat Loaf,Blind Before I Stop,i'll go blind before i stop [chorus] an...
3,Toto,Mrs. Johnson,know these words are wrong to say need you m...
4,Michael Bolton,Survivor,"(you're a survivor, you're a survivor, survivo..."


In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
stemmer = PorterStemmer()

In [ ]:
def token(txt):
  token= nltk.word_tokenize(txt)
  a=[stemmer.stem(w) for w in token]
  return " ".join(a)

In [ ]:
token("you are beautiful, beauty")

'you are beauti , beauti'

In [ ]:
data['text']= data['text'].apply(lambda x: token(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfid=TfidfVectorizer(analyzer='word', stop_words='english')

In [ ]:
matix=tfid.fit_transform(data['text'])

In [ ]:
similer=cosine_similarity(matix)

In [ ]:
similer[0]

array([1.        , 0.02246952, 0.0146776 , ..., 0.01204005, 0.07985522,
       0.01037806])

In [ ]:
def recommender(song_name):
  idx=data[data['song']==song_name].index[0]
  distance=sorted(list(enumerate(similer[idx])),reverse=True,key=lambda x:x[1])
  song=[]
  for s_id in distance[1:6]:
    song.append(data.iloc[s_id[0]].song)
  return song

In [ ]:
recommender("Autologic")

['A-tisket A-tasket',
 '100 Yard Dash',
 'My Vietnam',
 'The Bomb',
 'A Little Less Sixteen Candles, A Little']

In [ ]:
recommender("Blind Before I Stop")

["Doin' As I See On My Tv",
 'Whatever You Want Babe',
 'Cant Be Doin That Now',
 "I Ain't Got No Business Doin'",
 'Stuff Like That']

In [ ]:
import pickle

In [ ]:
pickle.dump(similer, open('similarity','wb'))

In [ ]:
pickle.dump(data, open('data','wb'))